# 콜금리 + CORPUS merge

In [10]:
callrate = pd.read_csv('20052017.csv',  index_col=0)

In [11]:
concat_ngram = pd.read_csv('concat_ngram.csv')

In [15]:
merge_file = concat_ngram.merge(callrate)
merge_file.head()

,date,body,ngram,price,up_and_down,label
0,2005-05-01,4월은 잔인했다. 어닝 시즌 기업들이 내놓은 실적은 실망감만 불렀고 세자리수대를 넘...,"[['기업/NNG;수익성/NNG;악화/NNG;우려/NNG', '실적/NNG;실망/N...",3.29,0.61%,up
1,2005-05-02,때이른 무더위로 5월 첫 주가 뜨겁다. 하지만 금융시장은 잔뜩 움츠린 모습니다....,"[['대외/NNG;불확실성/NNG', '절상/NNG'], ['성공/NNG', '최저...",3.29,0.00%,0
2,2005-05-03,3일 외환시장에서 환율이 상승 반전하고 있다. 이날 환율은 1000원으로 거래를 시...,"[['외환시장/NNG;환율/NNG;상승/NNG'], ['미/NNG;금리/NNG;인상...",3.30,0.30%,up
3,2005-05-04,앤디 시에 모건스탠리 아시아태평양 수석 이코노미스트는 4일 한국 경제 둔화세가 지속...,"[['경제/NNG;둔화/NNG'], ['v자형/NNG;회복/NNG', '소비/NNG...",3.21,-2.73%,down
4,2005-05-05,외환은행 은 오는 6일부터 전문직대상 신용대출 상품인 예스 프로론의 금리를 연 ...,"[['최저/NNG'], ['주택담보대출/NNG;금리/NNG;인하/NNG'], ['대...",3.21,1.87%,up


In [16]:
# 괄호, 공백 삭제
import re
tem = merge_file['ngram']
tem = [re.sub('[\[\]\']', "", i) for i in tem]
tem = [re.sub(' ', '', i) for i in tem]
merge_file['ngram'] = tem
merge_file.head()

,date,body,ngram,price,up_and_down,label
0,2005-05-01,4월은 잔인했다. 어닝 시즌 기업들이 내놓은 실적은 실망감만 불렀고 세자리수대를 넘...,"기업/NNG;수익성/NNG;악화/NNG;우려/NNG,실적/NNG;실망/NNG,우려/...",3.29,0.61%,up
1,2005-05-02,때이른 무더위로 5월 첫 주가 뜨겁다. 하지만 금융시장은 잔뜩 움츠린 모습니다....,"대외/NNG;불확실성/NNG,절상/NNG,성공/NNG,최저/NNG,급감/NNG,하락...",3.29,0.00%,0
2,2005-05-03,3일 외환시장에서 환율이 상승 반전하고 있다. 이날 환율은 1000원으로 거래를 시...,"외환시장/NNG;환율/NNG;상승/NNG,미/NNG;금리/NNG;인상/NNG,절상/...",3.30,0.30%,up
3,2005-05-04,앤디 시에 모건스탠리 아시아태평양 수석 이코노미스트는 4일 한국 경제 둔화세가 지속...,"경제/NNG;둔화/NNG,v자형/NNG;회복/NNG,소비/NNG;바닥/NNG,실망/...",3.21,-2.73%,down
4,2005-05-05,외환은행 은 오는 6일부터 전문직대상 신용대출 상품인 예스 프로론의 금리를 연 ...,"최저/NNG,주택담보대출/NNG;금리/NNG;인하/NNG,대출/NNG;금리/NNG;...",3.21,1.87%,up


In [27]:
merge_file.to_csv('merge_file.csv')
merge_file.to_pickle('merge_file.pkl')

In [ ]:
# 사전 구축

In [49]:
from collections import defaultdict
import pandas as pd 
import numpy as np
import math
import os
from os import listdir

doccnt1 = 0
doccnt0 = 0
wordfreq = defaultdict(lambda : [0, 0])
   
a = pd.read_csv('merge_file.csv')

#','을 기준으로 split
for nt, row in a.iterrows():
    words = str(row['ngram']).split(',')

    for word in words:
        if row['label'] == 'up':
            wordfreq[word][1] += 1
        else:
            wordfreq[word][0] += 1

a = pd.DataFrame.from_dict(wordfreq, orient='index')
a.to_csv("merge_lable_dict.csv")

In [50]:
a = pd.read_csv('merge_lable_dict.csv', names=['ngram', '비둘기파', '매파'])

In [51]:
a = a.dropna(axis=0)
a.to_csv("merge_lable_dict.csv")

In [52]:
a.head()

,ngram,비둘기파,매파
1,기업/NNG;수익성/NNG;악화/NNG;우려/NNG,7,6
2,실적/NNG;실망/NNG,143,34
3,우려/NNG;높/VV,166,73
4,둔화/NNG,9437,3242
5,하회/NNG,3478,1097


In [53]:
k =0.5
doccnt1 = 0
doccnt0 = 0

for key, (cnt0, cnt1) in wordfreq.items() :
    if cnt1 > 0 :
        doccnt1 += cnt1
    if cnt0 > 0 :
        doccnt0 += cnt0
        
wordprobs = defaultdict(lambda : [0,0])

for key, (cnt0, cnt1) in wordfreq.items() :
    wordprobs[key][1] = (cnt1 + k) / (doccnt1 + 2*k)   
    wordprobs[key][0] = (cnt0 + k) / (doccnt0 + 2*k)

b = pd.DataFrame.from_dict(wordprobs, orient='index')

b.to_csv("merge_lable_dict_prob.csv")

In [54]:
b = pd.read_csv("merge_lable_dict_prob.csv", header=None, names=['ngram', '비둘기파prob', '매파prob'])

In [55]:
b = b.dropna(axis=0)
b.to_csv("merge_lable_dict_prob.csv")

In [56]:
b.head()

,ngram,비둘기파prob,매파prob
1,기업/NNG;수익성/NNG;악화/NNG;우려/NNG,0.000003,0.000009
2,실적/NNG;실망/NNG,0.000063,0.000046
3,우려/NNG;높/VV,0.000074,0.000099
4,둔화/NNG,0.004168,0.004348
5,하회/NNG,0.001536,0.001472


In [57]:
a = pd.read_csv("./merge_lable_dict.csv")
b = pd.read_csv("./merge_lable_dict_prob.csv")

a['비둘기파prob'] = b["비둘기파prob"]
a['매파prob'] = b["매파prob"]

a.to_csv("./merge_lable_dict_prob.csv", index = False)

In [61]:
total = pd.read_csv("./merge_lable_dict_prob.csv")
tmp = total.sort_values('매파', ascending=False)
tmp = tmp.dropna(axis=0)
del tmp['Unnamed: 0']

In [63]:
tmp.head()

,ngram,비둘기파,매파,비둘기파prob,매파prob
23,하락/NNG,116998,37697,0.051666,0.050546
14,금리/NNG;인상/NNG,71669,24490,0.031649,0.032837
11,우려/NNG,34232,10997,0.015117,0.014746
76,금리/NNG;인하/NNG,38251,10607,0.016892,0.014223
5,최대/NNG,26982,8922,0.011915,0.011964


In [64]:
#tmp[tmp.ngram.str.contains(';')]
tmp['polarityscore'] = ''
tmp['polarityscore'] = tmp['매파prob'] /  tmp['비둘기파prob']
tmp.to_csv("DICT.csv" , index = False)

In [65]:
tmp.head()

,ngram,비둘기파,매파,비둘기파prob,매파prob,polarityscore
23,하락/NNG,116998,37697,0.051666,0.050546,0.978314
14,금리/NNG;인상/NNG,71669,24490,0.031649,0.032837,1.037551
11,우려/NNG,34232,10997,0.015117,0.014746,0.975442
76,금리/NNG;인하/NNG,38251,10607,0.016892,0.014223,0.841997
5,최대/NNG,26982,8922,0.011915,0.011964,1.004039


In [66]:
# Grey area 제거
b = tmp.loc[(tmp['polarityscore'] <= 1/1.3) | (tmp['polarityscore'] >= 1.3)]
b ['label'] = 0
b['label'] = [1 if x>1 else -1 for x in b['polarityscore']]
b.head()

C:\Users\hyeryeon\.conda\envs\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\hyeryeon\.conda\envs\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ngram,비둘기파,매파,비둘기파prob,매파prob,polarityscore,label
1172,콜/NNG;금리/NNG;인상/NNG,1479,1248,0.000653,0.001674,2.562239,1
232,비둘기/NNG,3816,955,0.001685,0.001281,0.760171,-1
102,인플레이션/NNG;우려/NNG,1628,742,0.000719,0.000996,1.384378,1
2700,fed/NNG;금리/NNG;인하/NNG,1280,632,0.000565,0.000848,1.499778,1
5244,통화/NNG;완화/NNG,2261,538,0.000999,0.000722,0.722995,-1


In [47]:
b ['label'] = 0
b['label'] = [1 if x>1 else -1 for x in b['polarityscore']]
b.to_csv("FINAL_DICT.csv", index = False)

C:\Users\hyeryeon\.conda\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\hyeryeon\.conda\envs\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
